In [100]:
# Importar librerias
# Cargar las API keys necesarias
# Hacer las conexiones necesarias
#!pip install -U langchain-community


import os
import json
import chromadb # Biblioteca para gestionar bases de datos de vectores y para trabajar con embeddings.
import cohere   # Cliente para interactuar con los modelos LLM de Cohere.
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output
from langchain.text_splitter import RecursiveCharacterTextSplitter # Se utiliza para dividir el texto en fragmentos más pequeños (chunks) respetando un tamaño máximo y un solapamiento definido.
from PyPDF2 import PdfReader  # Se emplea para leer y extraer el texto de archivos PDF.
from langchain.embeddings import CohereEmbeddings  # Clase para trabajar con embeddings generados por Cohere.
from langchain.vectorstores import Chroma # Integra almacenamiento vectorial con embeddings para búsquedas eficientes.
from langchain.document_loaders import TextLoader  # Carga documentos de texto para su procesamiento posterior.


#!pip install langchain PyPDF2
#!pip install chromadb langchain





load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
#print(api_key)  # Verify the key is loaded


# Establezco la conexion a cohere para el embedding y hago la primer consulta con el modelo de embeddings a modo de prueba.
# Uso la api version v2

co = cohere.ClientV2()
response = co.embed(
    texts=["hola"],
    model="embed-multilingual-v3.0",
    input_type="search_document",
    embedding_types=["float"],
)


#print(response)
#print(response.embeddings.float_[0])
#print(len(response.embeddings.float_[0]))


# Establezco la conexion a cohere y realizo una primer consulta a algun modelo del LLM a modo de prueba.
# Utilizo la api version v2

co = cohere.ClientV2()
response = co.chat(
    model="command-r-plus",
    messages=[{"role": "user", "content": "¿Qué país ganó el mundial 2022?"}],
)

#print(response.message.content[0].text)


In [94]:
# Carga de los documentos 

In [101]:
from PyPDF2 import PdfReader # Se utiliza para leer y extraer texto de archivos PDF.
from langchain.text_splitter import RecursiveCharacterTextSplitter # Herramienta para dividir texto en fragmentos de tamaño manejable.

# Cargar el contenido del PDF
def cargar_pdf(ruta_pdf):
    """
    Carga el contenido de un archivo PDF y extrae su texto.
    
    Args:
        ruta_pdf (str): Ruta relativa del archivo PDF.
    
    Returns:
        str: Texto completo extraído del PDF.
    """
    reader = PdfReader(ruta_pdf)
    texto = ""
    for page in reader.pages:              # Itera sobre todas las páginas del PDF.
        texto += page.extract_text()       # Extrae el texto de cada página y lo agrega al texto completo.
    return texto

# Dividir el texto por historias
def dividir_por_historias(texto):
    """
    Divide el texto en historias usando títulos identificados.
    
    Args:
        texto (str): Texto completo del PDF.
    
    Returns:
        dict: Diccionario con el título de la historia como clave y su texto como valor.
    """
    separadores = ["Sol y Luna", "La tortuga Tica", "El Duende"]
    historias = {}
    for i, titulo in enumerate(separadores):
        inicio = texto.find(titulo)  # Encuentra la posición del título actual
        fin = texto.find(separadores[i + 1]) if i + 1 < len(separadores) else len(texto) # Encuentra la posición del siguiente título o el final del texto.
        historias[titulo] = texto[inicio:fin].strip()  # Extrae el texto correspondiente a la historia y elimina espacios adicionales.
    return historias # Retorna un diccionario con las historias separadas.


# Aplicar chunking a cada historia
def aplicar_chunking_a_historias(historias, chunk_size=300, chunk_overlap=100):
    """
    Aplica chunking a cada historia individualmente con chunks pequeños.
    
    Args:
        historias (dict): Diccionario con historias separadas.
        chunk_size (int): Longitud máxima de cada chunk.
        chunk_overlap (int): Superposición entre chunks.
    
    Returns:
        dict: Diccionario con chunks de cada historia.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap) # Configura el divisor de texto con tamaño máximo y superposición
    chunks_por_historia = {}
    for titulo, texto in historias.items():
        chunks = splitter.split_text(texto)  # Divide el texto de la historia en chunks.
        chunks_por_historia[titulo] = chunks # Asocia los chunks al título de la historia.
    return chunks_por_historia  # Retorna un diccionario con los chunks por historia.

# Ruta del archivo PDF
ruta_pdf = "Historias.pdf"

# Cargar el PDF
texto_pdf = cargar_pdf(ruta_pdf)

# Dividir el texto por historias
historias = dividir_por_historias(texto_pdf)

# Aplicar chunking a cada historia (ajustado a más chunks) individualmente.
chunks_por_historia = aplicar_chunking_a_historias(historias)  

# Mostrar los resultados
for titulo, chunks in chunks_por_historia.items():
    print(f"Historia: {titulo}")
    print(f"Cantidad de chunks generados: {len(chunks)}")
    for i, chunk in enumerate(chunks[:3]):  # Mostrar los primeros 3 chunks por historia como ejemplo
        print(f"Chunk {i+1}:\n{chunk}\n{'-'*50}")



# Nota: Optamos por la técnica de chunking con un tamaño de chunk de 300 caracteres y una superposición de 100 porque:
# - El tamaño de chunk (300) es lo suficientemente pequeño para asegurar que cada fragmento sea procesable y manejable por modelos de lenguaje o sistemas de búsquedasda.
# - La superposición (100) garantiza que las ideas no se corten abruptamente entre chunks, manteniendo el contexto entre fragmentos consecutivos.




Historia: Sol y Luna
Cantidad de chunks generados: 13
Chunk 1:
Sol y Luna 
Sol y Luna eran dos pequeños gatitos que habían nacido en la misma camada, pero sus 
personalidades no podían ser más diferentes. Sol, de un brillante color anaranjado, era 
aventurero y curioso; siempre buscando nuevas experiencias y explorando cada rincón de
--------------------------------------------------
Chunk 2:
aventurero y curioso; siempre buscando nuevas experiencias y explorando cada rincón de 
su hogar. Luna, en cambio, era de un suave pelaje gris y tenía un temperamento más 
sereno y observador. Pasaba horas contemplando el mundo desde la ventana, como si en
--------------------------------------------------
Chunk 3:
sereno y observador. Pasaba horas contemplando el mundo desde la ventana, como si en 
cada sombra descubriera un misterio oculto.  
Una tarde, mientras Sol correteaba por el jardín persiguiendo mariposas, Luna permanecía
--------------------------------------------------
Historia: La to

In [106]:
# Conexion a la base de datos


In [107]:
# Borrar la colección si ya existe
chroma_client.delete_collection(name="historias_chroma")

In [108]:
# Carga de datos en la base
# import chromadb (ya se importó en la primera celda, pero lo dejo comentado para entender que aquí se utiliza. Esto aplica para los demás)
# import json

# Conectar a Chroma
# Si se desea almacenar la base de datos en el disco local
chroma_client = chromadb.PersistentClient(path="C:/Users/Trini/OneDrive - frc.utn.edu.ar/Escritorio/PI CONSULTING/Get Talent/PracticaSemana3")

# Levantar la base o crear una colección si no existe
collection = chroma_client.get_or_create_collection(name="historias_chroma")

# Cargar los chunks generados previamente
# con los títulos de las historias como claves y listas de chunks como valores.
chunks = []
ids = []

# Procesar los chunks generados
for titulo, lista_chunks in chunks_por_historia.items(): # Itera sobre cada historia y sus chunks asociados
    for i, chunk in enumerate(lista_chunks): # Itera sobre los chunks individuales de una historia
        chunks.append(chunk)  # Agrega el chunk a la lista de documentos.
        ids.append(f"{titulo}_chunk_{i}")    # Crea un identificador único para el chunk basado en el título y su índice.


# Cargar los documentos en la base de datos
collection.add(
    documents=chunks,
    ids=ids
)

# Realizar consulta sobre la base usando texto
query_text = "¿Donde vivia la tortuga Tica ?..."
results_text_query = collection.query(
    query_texts=[query_text],  # Chroma generará los embeddings de esta consulta
    n_results=2  # Número de resultados a devolver
)

# Mostrar los resultados de la consulta basada en texto
print("Resultados de la consulta basada en texto:")
print(json.dumps(results_text_query, indent=2))


# Nota: Al no usar embeddings personalizados, no se utiliza un modelo externo como Cohere.


Resultados de la consulta basada en texto:
{
  "ids": [
    [
      "La tortuga Tica_chunk_0",
      "Sol y Luna_chunk_10"
    ]
  ],
  "embeddings": null,
  "documents": [
    [
      "La tortuga Tica \nUna tortuga llamada Tica viv\u00eda en un tranquilo estanque rodeado de \u00e1rboles frondosos. A \ndiferencia de sus compa\u00f1eras, Tica so\u00f1aba con explorar m\u00e1s all\u00e1 del agua tranquila y la",
      "mientras Luna lo vigilaba con ojos atentos. En ese momento, comprendieron que au nque \neran diferentes como el d\u00eda y la noche, siempre estar\u00edan ah\u00ed el uno para el otro. Su v\u00ednculo \nera m\u00e1s fuerte que cualquier tormenta."
    ]
  ],
  "uris": null,
  "data": null,
  "metadatas": [
    [
      null,
      null
    ]
  ],
  "distances": [
    [
      0.6646159683326235,
      0.9243467262998977
    ]
  ],
  "included": [
    "distances",
    "documents",
    "metadatas"
  ]
}
Resultados de la consulta basada en texto (segunda consulta):
{
  "ids": [

In [75]:
# Prompt

# Incluir personalidad y especificaciones de las respuestas.
# Incluir la pregunta del usuario
# Incluir la informacion devuelta por la base de datos





In [110]:
## Bloque importacion de librerias

# import json
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# bloque variables de entorno
# from dotenv import load_dotenv
# import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded

## bloque conexion a Cohere
import cohere

co = cohere.ClientV2()

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)


r3eUg1U6gFJAeIuUddWn8QAFRXpwEYmEP71eXwZO
id='57d685d4-fdf1-4f4f-867d-cc67ea11f8a7' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


In [77]:
# RAG Integration

In [111]:

def RAG_answer(question):
    # Consultar los documentos relevantes
    results = collection.query(
        query_texts=[question], # Realiza una consulta en la base de datos utilizando la pregunta como entrada.
        n_results=3 # Devuelve los 3 documentos más relevantes según los embeddings
    )

    # Combinar fragmentos de texto para cada documento
    context = "\n".join([" ".join(doc) for doc in results["documents"]])

    
    # Crear el prompt para el LLM
    system_prompt = """
Eres un asistente experto en responder preguntas basándote exclusivamente en la información contextual proporcionada.

Reglas:
- Responde de manera amigable y entusiasta, como si hablaras con un niño.
- Responde en máximo 3 oraciones.
- Agrega emojis a tu respuesta.
- Responde siempre en español, sin importar en qué idioma se haga la pregunta.
- Solo utiliza el contenido proporcionado en el contexto para responder.

Si no puedes responder usando el contexto, di: 
"Lo siento, no puedo responder esa pregunta con la información proporcionada. 🌟".
    """

    user_prompt = f"""
Contexto relevante:
{context}

Pregunta:
{question}

Respuesta:
    """
    # Estructura el prompt que se enviará al modelo, incluyendo el contexto relevante y la pregunta del usuario.
    
    # Llamar al modelo LLM
    response = co.chat(
        model="command-r-plus-08-2024",
        messages=[
            {"role": "system", "content": system_prompt}, # El sistema define el comportamiento del modelo.
            {"role": "user", "content": user_prompt},  # El usuario envía la pregunta junto con el contexto relevante
        ],
        temperature=0.7 # Controla la creatividad del modelo en las respuestas (0 = más determinista, 1 = más creativo).
    )

    # Devolver la respuesta
    return response.message.content[0].text # Extrae y retorna el texto generado por el modelo


In [112]:
question = "¿Quién es Luna?"
respuesta = RAG_answer(question)
print(respuesta)


🐈 Luna es la gatita hermana de Sol, una compañera de aventuras y muy protectora. ¡Ella es como la luz de la luna, guiando a su hermano en momentos de oscuridad! 🌙🐾


In [81]:
question = "¿De qué color es Sol?"
respuesta = RAG_answer(question)
print(respuesta)

¡Hola! 😺 Sol es de un hermoso color anaranjado, como el atardecer. 🌞🧡 ¿No es genial que los gatitos puedan tener colores tan bonitos?


In [82]:
question = "¿Cómo se llama el duende?"
respuesta = RAG_answer(question)
print(respuesta)

¡El duende se llama Puck! 🧚‍♂️ Un nombre muy divertido, ¿verdad? 😄


In [83]:
question = "¿De qué club es hincha la tortuga Tica?"
respuesta = RAG_answer(question)
print(respuesta)

Lo siento, no puedo responder esa pregunta con la información proporcionada. 🌟 ¿Quieres saber algo más sobre Tica o los gatitos? 😊


In [85]:
question = "¿Cual es la enseñanza del cuento 'El duende'?"
respuesta = RAG_answer(question)
print(respuesta)

¡Vaya, qué cuento tan divertido! 🧚‍♂️ Puck aprendió que las bromas son geniales, pero hay que ser amable y pensar en los sentimientos de los demás. ¡Una gran lección para un duende travieso! 😁🌿


In [86]:
question = "¿Cual es la enseñanza del cuento 'El duende'?"
respuesta = RAG_answer(question)
print(respuesta)


¡Hola, amigo! 😊 La historia de Puck nos enseña que debemos pensar en los sentimientos de los demás, incluso cuando queremos divertirnos. 🧚‍♀️ ¡Las bromas son geniales, pero la amabilidad es aún mejor! 🌈


In [88]:
question = "What is turttle's name?"
respuesta = RAG_answer(question)
print(respuesta)

¡La tortuga se llama Tica! 🐢 Un nombre muy lindo, ¿no crees?


In [92]:
question = "Qual é o nome do duende?"
respuesta = RAG_answer(question)
print(respuesta)

¡Hola! El nombre del duende es Puck. 🧚‍♂️ Un nombre muy divertido, ¿no crees? 😄
